**POC for identifying groups of individuals who can live together at a new dwelling**


In [167]:
# imports
import os
import pandas as pd
import numpy as np

In [168]:
# load in CSV of Persons
# expect single CSV to have all necessary info about a person
people_filepath = 'data/PeopleSheet.csv'
peeps = pd.read_csv(people_filepath)

In [169]:
# Create a Dwelling to test People against
# Note: no geo math, just flag whether, e.g., it is no_offenders (True)
dwelling_filepath = 'data/DwellingSheet.csv'
dwelling = pd.read_csv(dwelling_filepath)

In [170]:
# Test Persons against Dwelling, return list of eligible and eliminated
if not dwelling.offender_ok[0] == 'y':
    eligibles_loc = peeps[peeps['is_offender'] == 'n']  ## fragile: uppercase breaks
    eliminated_loc = peeps[peeps['is_offender'] == 'y']

print(f'# eligible: {len(eligibles_loc)}')
print(f'# eliminated: {len(eliminated_loc)}')

if len(eligibles_loc) + len(eliminated_loc) == len(peeps):
    print(f'All {len(peeps)} peeps accounted for!')    
else:
    print('Looks like we might have lost one! Started with X peeps.')
# test that all records went one place or the other (using .len?)

# eligible: 6
# eliminated: 1
All 7 peeps accounted for!


**HARD GROUPING**

Hard-group eligible persons based on a small number of lifestyle flags

**Hard grouping within DF**

This approach utilizes the existing dataframe for the grouping analysis. It "adds" individuals to groups one at a time by assigning them a value for the 'group' column.

In [171]:
# add 'group' column to peeps, initialize to 0
# This is its own block because it will error out if you add the same col twice.
eligibles_loc.insert(1, 'hard_group', -1)  

In [177]:
flag_criterion = 'smoking'

for index, peep in eligibles_loc.iterrows():
    check_group = 0 # for each peep, check fit starting with group 0
    # any groups left to test fit with? If not, must need another group for this peep
    if check_group > eligibles_loc.hard_group.max():
        print(f'new group please')
        eligibles_loc.hard_group[eligibles_loc.UID == peep.UID] = check_group
    # otherwise, see if peep is compatible with members of current group, by iterating through group
    else:
        for index, groupie in eligibles_loc[eligibles_loc.hard_group == check_group].iterrows():
            if groupie[flag_criterion + '_has'] == 'y' and peep[flag_criterion + '_conflict'] == 'y':
                check_group = check_group + 1
                break
        # if it didn't 'break', the group fit is okay; so assign peep to this group
        eligibles_loc.hard_group[eligibles_loc.UID == peep.UID] = check_group
             
            



#     for checkgroup in range(len(peeps[
    # iterate groups
    
    # iterate peeps in group, check flags for new person
# for index, peep in eligibles_loc.iterrows():
#     print(peep.UID, peep.hard_group)


/Users/wes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/wes/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/wes/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [178]:
eligibles_loc

,UID,hard_group,is_offender,smoking_has,smoking_conflict,dog_has,dog_conflict
1,X0002,0,n,y,n,y,n
2,X0003,0,n,y,n,y,n
3,X0004,0,n,y,n,y,n
4,X0005,1,n,y,y,y,n
5,X0006,0,n,y,n,y,n
6,X0007,1,n,n,y,y,y


In [179]:
# Report groups
